In [1]:
import ast
import pandas as pd
import numpy as np
from pylogit import choice_tools
import ast
import pandas as pd
import numpy as np
from pylogit import choice_tools
from sklearn.model_selection import train_test_split

**Step 1: Covert the Files into a database**

In [2]:
#Function to read a file where each line is a Python list literal
def read_list_literal_file(file_path):
    with open(file_path, 'r') as file:
        
        data = [ast.literal_eval(line.strip()) for line in file]
    return data

# Function to read a file where each line is a float number
def read_float_file(file_path):
    with open(file_path, 'r') as file:
        data = [(line.strip()) for line in file]
    return data

# Read the data from the files
assortment_data = read_list_literal_file('assortment.txt')
probability_data = read_float_file('probability.txt')

# Convert the data into pandas DataFrame
df = pd.DataFrame({
    'Assortment': assortment_data,
    'Probability': probability_data
})

# Save the DataFrame to a CSV file
df.to_csv('clean_train.csv', index=False)
df.head()

,Assortment,Probability
0,"[0, 2]","[0.5796182233807586, 0.42038177661924164]"
1,"[0, 7, 14]","[0.1391774481903281, 0.40211566218661293, 0.45..."
2,"[0, 3, 30, 23]","[0.3584641056553135, 0.1281686726296265, 0.251..."
3,"[0, 5, 21, 20, 7]","[0.10945075272944618, 0.06742039394070407, 0.0..."
4,"[0, 23, 2, 19, 11, 28]","[0.20553684973140704, 0.0846429998294485, 0.05..."


**Step 2 : Simulate the choice data: assume the product with the highest probability in each assortment was chosen**

In [3]:
import ast
df_clean_train = pd.read_csv('clean_train.csv')
# Convert the string representations of lists into actual lists
df_clean_train['Assortment'] = df_clean_train['Assortment'].apply(ast.literal_eval)
df_clean_train['Probability'] = df_clean_train['Probability'].apply(ast.literal_eval)

# Expand the DataFrame to a long format
rows = []
for _, row in df_clean_train.iterrows():
    for product, probability in zip(row['Assortment'], row['Probability']):
        rows.append({'Product': product, 'Probability': probability})

# Create a new DataFrame
df_long_format = pd.DataFrame(rows)

# Simulate the choice data: assume the product with the highest probability in each assortment was chosen
# For simplicity, we'll assume the product index corresponds to the SKU ID in the assortments
df_long_format['Chosen'] = 0  # Initialize all as not chosen
# Get the index of the max probability in each assortment and set as chosen
for idx, group in df_clean_train.iterrows():
    chosen_idx = group['Probability'].index(max(group['Probability']))
    chosen_product = group['Assortment'][chosen_idx]
    # Set the chosen product in the long format dataframe
    df_long_format.loc[(df_long_format['Product'] == chosen_product) & (df_long_format.index == idx), 'Chosen'] = 1
#1 and 0 got dog problem
# Correcting the logic to mark 'Chosen' products based on the highest probability within each assortment group

# First, we need to identify each assortment group. We'll add an 'Assortment_ID' to df_long_format to do this.
df_long_format['Assortment_ID'] = df_clean_train.index.repeat(df_clean_train['Assortment'].str.len()).values

# Now, we can group by 'Assortment_ID' and mark the product with the highest probability as chosen
df_long_format['Chosen'] = 0  # Reset all to not chosen

for assortment_id, group in df_long_format.groupby('Assortment_ID'):
    # Find the index of the row with the max probability in the group
    idx_max_prob = group['Probability'].idxmax()
    # Set 'Chosen' to 1 for this product
    df_long_format.at[idx_max_prob, 'Chosen'] = 1

df_long_format['Cores'] = 0 
df_long_format['Frequency'] = 0 
df_long_format['TDP'] = 0 
df_long_format['Price'] = 0 
# Let's take a look at the resulting long format DataFrame
df_long_format.head()

,Product,Probability,Chosen,Assortment_ID,Cores,Frequency,TDP,Price
0,0,0.579618,1,0,0,0,0,0
1,2,0.420382,0,0,0,0,0,0
2,0,0.139177,0,1,0,0,0,0
3,7,0.402116,0,1,0,0,0,0
4,14,0.458707,1,1,0,0,0,0


**Step 3 Mapping of data base on the context**

In [ ]:
# Create price levels for each product
prices_all_products = [1800, 3000, 2700, 2400, 2100]
cores = [4, 8, 8, 4, 4, 4]
freq = [3.2, 2.9, 2.9, 2.9, 3.2, 3.2]
TDP = [95, 60, 95, 60, 60, 135]

# Reshape data for MNLogit
for i, rows in df_long_format.iterrows():
    x = df_long_format['Product'][i]
    if x == 0:
        df_long_format.loc[i,'Cores'] = 0
        df_long_format.loc[i,'Frequency'] = 0
        df_long_format.loc[i,'TDP'] = 0
        df_long_format.loc[i,'Price'] = 0
    elif x <6:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 3.2
        df_long_format.loc[i,'TDP'] = 95
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 11:
        df_long_format.loc[i,'Cores'] = 8
        df_long_format.loc[i,'Frequency'] = 2.9
        df_long_format.loc[i,'TDP'] = 60
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 16:
        df_long_format.loc[i,'Cores'] = 8
        df_long_format.loc[i,'Frequency'] = 2.9
        df_long_format.loc[i,'TDP'] = 95
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 21:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 2.9
        df_long_format.loc[i,'TDP'] = 60
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 26:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 3.2
        df_long_format.loc[i,'TDP'] = 60
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    else:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 2.2
        df_long_format.loc[i,'TDP'] = 135
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]

mnl_data2 = pd.DataFrame(df_long_format)

mnl_data2.head()

C:\Users\mi\AppData\Local\Temp\ipykernel_17496\2404755099.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_long_format.loc[i,'Frequency'] = 3.2


**Step 4: Split the original training data to clean_train and clean_test**

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the DataFrame to a CSV file
train_df.to_csv('clean_train.csv', index=False)

test_df.to_csv('clean_test.csv', index=False)


**Step 5: Convert clean_train to clean_train_long**

In [ ]:
import ast
df_clean_train = pd.read_csv('clean_train.csv')
# Convert the string representations of lists into actual lists
df_clean_train['Assortment'] = df_clean_train['Assortment'].apply(ast.literal_eval)
df_clean_train['Probability'] = df_clean_train['Probability'].apply(ast.literal_eval)

# Expand the DataFrame to a long format
rows = []
for _, row in df_clean_train.iterrows():
    for product, probability in zip(row['Assortment'], row['Probability']):
        rows.append({'Product': product, 'Probability': probability})

# Create a new DataFrame
df_long_format = pd.DataFrame(rows)

# Simulate the choice data: assume the product with the highest probability in each assortment was chosen
# For simplicity, we'll assume the product index corresponds to the SKU ID in the assortments
df_long_format['Chosen'] = 0  # Initialize all as not chosen
# Get the index of the max probability in each assortment and set as chosen
for idx, group in df_clean_train.iterrows():
    chosen_idx = group['Probability'].index(max(group['Probability']))
    chosen_product = group['Assortment'][chosen_idx]
    # Set the chosen product in the long format dataframe
    df_long_format.loc[(df_long_format['Product'] == chosen_product) & (df_long_format.index == idx), 'Chosen'] = 1
#1 and 0 got dog problem
# Correcting the logic to mark 'Chosen' products based on the highest probability within each assortment group

# First, we need to identify each assortment group. We'll add an 'Assortment_ID' to df_long_format to do this.
df_long_format['Assortment_ID'] = df_clean_train.index.repeat(df_clean_train['Assortment'].str.len()).values

# Now, we can group by 'Assortment_ID' and mark the product with the highest probability as chosen
df_long_format['Chosen'] = 0  # Reset all to not chosen

for assortment_id, group in df_long_format.groupby('Assortment_ID'):
    # Find the index of the row with the max probability in the group
    idx_max_prob = group['Probability'].idxmax()
    # Set 'Chosen' to 1 for this product
    df_long_format.at[idx_max_prob, 'Chosen'] = 1

df_long_format['Cores'] = 0 
df_long_format['Frequency'] = 0 
df_long_format['TDP'] = 0 
df_long_format['Price'] = 0 
# Let's take a look at the resulting long format DataFrame
df_long_format.head()

In [ ]:
# Create price levels for each product
prices_all_products = [1800, 3000, 2700, 2400, 2100]
cores = [4, 8, 8, 4, 4, 4]
freq = [3.2, 2.9, 2.9, 2.9, 3.2, 3.2]
TDP = [95, 60, 95, 60, 60, 135]

# Reshape data for MNLogit
for i, rows in df_long_format.iterrows():
    x = df_long_format['Product'][i]
    if x == 0:
        df_long_format.loc[i,'Cores'] = 0
        df_long_format.loc[i,'Frequency'] = 0
        df_long_format.loc[i,'TDP'] = 0
        df_long_format.loc[i,'Price'] = 0
    elif x <6:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 3.2
        df_long_format.loc[i,'TDP'] = 95
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 11:
        df_long_format.loc[i,'Cores'] = 8
        df_long_format.loc[i,'Frequency'] = 2.9
        df_long_format.loc[i,'TDP'] = 60
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 16:
        df_long_format.loc[i,'Cores'] = 8
        df_long_format.loc[i,'Frequency'] = 2.9
        df_long_format.loc[i,'TDP'] = 95
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 21:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 2.9
        df_long_format.loc[i,'TDP'] = 60
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    elif x < 26:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 3.2
        df_long_format.loc[i,'TDP'] = 60
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]
    else:
        df_long_format.loc[i,'Cores'] = 4
        df_long_format.loc[i,'Frequency'] = 2.2
        df_long_format.loc[i,'TDP'] = 135
        df_long_format.loc[i,'Price'] = prices_all_products[x%5]

In [ ]:
mnl_data = pd.DataFrame(df_long_format)

mnl_data.head()

**Step 6: Consolidation of Files**

In [ ]:



mnl_data2.to_csv('Trainingdata2.csv', index=False) #This is the original data set 


mnl_data.to_csv('clean_train_long.csv', index=False) # this is 0.8 of the training set. the other 0.2 is for validation.